# Play Generator

In [ ]:
%tensorflow_version 2.x

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

The dataset here is going to be Romeo and Juliet, but, if we would like, we could use any, even our own write ups and poems.

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


# Loading Your Own Data



In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]  # note, whatever file you choose, should be a txt file.

# Reading the File

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


# Encoding

Carrying out some text preprocessing.

In [ ]:
vocab = sorted(set(text)) # This will basically sort out all the unique characters in the play
char_to_index = {u:i for i, u in enumerate(vocab)}
index_to_char = np.array(vocab)

def text_to_int(text):
  return np.array([char_to_index[c] for c in text]) #for each character, which represents a key in the dictionary, it will return its value(index) after converting them all to an array

text_as_int = text_to_int(text)

A simple functin to convert from the integer encoding back to the text

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy() #Converting to a numpy array, if this isn't already one
  except:
    pass # if it is, then don't do anything
  return ''.join(index_to_char[ints])

# Creating Training Examples

We have to split the text into shorter sequences to pass to the model as training examples, since our model is going to be returning 1 character per training example.

This training example will use a seq_length sequence as input and outpit, where the one at the output is the original sequence shifted one letter to the right.

The first step will be to create a stream of characters from our text data

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1) #Plus 1 because the output sequence will be shifted by 1 to the right, hence it is actually 101 characters used

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)   # make the string a stream of characters

Now we turn the stream of characters into the batches of desired length

In [ ]:
sequences = char_dataset.batch(seq_length + 1, drop_remainder = True) # any remainder after the division that is not enough to be batched, is dropped

The sequences of length 101 will be split into input and output

In [ ]:
def split_input_target(full):
  input_text = full[:-1]
  target_text = full[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target) # This will apply the function to every sequence

Then we make our training batches

In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMENSIONS = 256 # This describes the vector dimensions in the embedding layer
RNN_UNITS = 1024

BUFFER_SIZE = 10000
data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

# Building the Model

The dense layer in our model will contain a node for each unique character in our training data. The dense layer will give us a probability distribution over each node.

In [ ]:
def build_model(vocab_size, embedding_dimensions, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dimensions, 
                                                         batch_input_shape = [batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units, return_sequences = True, #This is so that the model will return the output at every timestamp, not just the final output
                                                    stateful = True, recurrent_initializer = 'glorot_uniform'),  #LSTM is long short term memory
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIMENSIONS, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


# Creating a Loss Function

We need to create a loss function, because of the type of predictions our model will be making. First, we are looking at an input and the output from our untrained model, just to see what the model is giving us

In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # Ask our model for a prediction on our first batch.
  print(example_batch_predictions.shape, ' = (batch_size, sequence_length, vocab_size)')

pred = example_batch_predictions[0]

(64, 100, 65)  = (batch_size, sequence_length, vocab_size)


In [ ]:
# If we want to determine the predicted character, we need to sample the output distribution (pick a value based on probability distribution)
sampled_indices = tf.random.categorical(pred, num_samples = 1)  #We're sampling

# Now, we reshape the array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars

"\nIWitI:H-IakG,gsX UnxkRSBXmXsNyhgcMzb'VqXU,aTaEy?bfKFQRuo,&fMSPUt:QUYamweq$z;yeJ.V:K$UxW3FgNt -,!koA"

Now we create a loss function that can compare the output to the expected output and give us some numeric value representing how close the two were

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

# Compiling the Model

In [ ]:
model.compile(optimizer = 'adam', loss = loss)

# Creating Checkpoints

Now we are going to setup and configure our model to save checkpoints as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'   
#Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

# Training

In [ ]:
history = model.fit(data, epochs =100, callbacks = [checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 15s 68ms/step - loss: 2.5771
Epoch 2/100
172/172 [==============================] - 13s 69ms/step - loss: 1.8658
Epoch 3/100
172/172 [==============================] - 13s 71ms/step - loss: 1.6189
Epoch 4/100
172/172 [==============================] - 13s 69ms/step - loss: 1.4898
Epoch 5/100
172/172 [==============================] - 13s 69ms/step - loss: 1.4128
Epoch 6/100
172/172 [==============================] - 13s 69ms/step - loss: 1.3575
Epoch 7/100
172/172 [==============================] - 13s 69ms/step - loss: 1.3123
Epoch 8/100
172/172 [==============================] - 13s 69ms/step - loss: 1.2723
Epoch 9/100
172/172 [==============================] - 13s 68ms/step - loss: 1.2361
Epoch 10/100
172/172 [==============================] - 13s 69ms/step - loss: 1.1988
Epoch 11/100
172/172 [==============================] - 13s 69ms/step - loss: 1.1623
Epoch 12/100
172/172 [==============================] - 13s 69ms/step - lo

# Loading the Model

We'll rebuild the model from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIMENSIONS, RNN_UNITS, batch_size = 1)

Once the model is finished, we can find the latest checkpoint that stores the models weights using the following line

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load any checkpoint we want to by specifying the exact file to load

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint('./training_checkpoints/ckpt_' + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

# Generate Text

In [ ]:
def generate_text(model, start_string):
  num_generate = 1000 #Number of characters to generate

  input_eval = text_to_int(start_string)
  input_eval = tf.expand_dims(input_eval, 0)  # This adds an extra nested list on it, since thats what the model is expecting\

  text_generated = [] #empty string to store our results

  # Low temperatures results in more predictible text, while higher ones result in surprising text
  # Experiment to find the best setting
  temperature = 1.0

  model.reset_states() # While training, the model will have saved its states, we don't want that so we clear those saved states

  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0) # removes the extra dimension we added

    predictions = predictions/temperature
    predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy() # Sampling

    input_eval = tf.expand_dims([predicted_id], 0) # We are passing the predicted character as the next input into the model

    text_generated.append(index_to_char[predicted_id]) # appending the output to the empty list

  return(start_string + ''.join(text_generated))
  

In [ ]:
inp = input('Type a starting string: ')
print(generate_text(model, inp))